In [ ]:
import requests
import pandas as pd
import os
from urllib.parse import urlparse
from pathlib import Path

# Base URL
base_url = "https://purpink.co.ke/collections/gifts-for-her/products.json"

# Folder to store images
images_folder = "images"
os.makedirs(images_folder, exist_ok=True)

# Store all variant records
all_products = []

# Start pagination
page = 1
while True:
    print(f"🔄 Fetching page {page}...")
    response = requests.get(f"{base_url}?page={page}")

    if response.status_code != 200:
        print("❌ Failed to fetch data. Stopping.")
        break

    products = response.json().get("products", [])
    if not products:
        print("✅ No more products to fetch.")
        break

    for p in products:
        title = p.get("title")
        handle = p.get("handle")
        link = f"https://purpink.co.ke/products/{handle}"

        # Try to get first image
        img_url = None
        if p.get("images"):
            img_url = p["images"][0].get("src")

            # ✅ Download image if available
            if img_url:
                img_filename = Path(urlparse(img_url).path).name
                img_path = os.path.join(images_folder, img_filename)

                if not os.path.exists(img_path):  # Avoid re-downloading
                    img_data = requests.get(img_url).content
                    with open(img_path, "wb") as f:
                        f.write(img_data)
                    print(f"🖼️ Downloaded: {img_filename}")

        # Loop through variants
        for variant in p.get("variants", []):
            variant_title = variant.get("title")
            sku = variant.get("sku", "")
            price = variant.get("price")

            all_products.append({
                "Product Title": title,
                "Variant": variant_title,
                "Price": f"KSh {price}",
                "SKU": sku,
                "Product Link": link,
                "Image URL": img_url,
                "Image File": img_filename if img_url else None
            })

    page += 1

# ✅ Save results
df = pd.DataFrame(all_products)
df.to_csv("purpink_gifts_variants.csv", index=False)
print(f"\n📁 Done! {len(df)} product variants saved to purpink_gifts_variants.csv")
